# Proje: Kripto Para Fiyat Tahmini (Cryptocurrency Price Prediction)

Bu görev için, yfinance API'sini kullanarak Yahoo Finance'den en son Bitcoin fiyat verilerini toplayacağız. Bu, bu kodu her çalıştırdığınızda en son verileri toplamanıza yardımcı olacaktır.

In [1]:
pip install yfinance

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import yfinance as yf
import datetime
from datetime import date, timedelta
today = date.today()

d1 = today.strftime("%Y-%m-%d")
end_date = d1
d2 = date.today() - timedelta(days=730)
d2 = d2.strftime("%Y-%m-%d")
start_date = d2

data = yf.download('BTC-USD', 
                      start=start_date, 
                      end=end_date, 
                      progress=False)
data["Date"] = data.index
data = data[["Date", "Open", "High", "Low", "Close", "Adj Close", "Volume"]]
data.reset_index(drop=True, inplace=True)
data.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2022-10-26,20092.236328,20938.134766,20076.117188,20770.441406,20770.441406,58895950537
1,2022-10-27,20772.802734,20854.044922,20255.373047,20285.835938,20285.835938,49625110402
2,2022-10-28,20287.957031,20724.980469,20086.068359,20595.351562,20595.351562,43994715910
3,2022-10-29,20595.103516,20988.394531,20566.484375,20818.476562,20818.476562,40369840645
4,2022-10-30,20817.982422,20917.005859,20547.462891,20635.603516,20635.603516,31486345556


Yukarıdaki kodda, son 730 güne ait Bitcoin fiyatlarının en son verilerini topladım ve ardından herhangi bir veri bilimi görevi için hazırladım. Şimdi, 730 satırla çalışıp çalışmadığımızı görmek için bu veri kümesinin şekline bir bakalım

In [4]:
data.shape

(729, 7)

In [5]:
import plotly.graph_objects as go
figure = go.Figure(data=[go.Candlestick(x=data["Date"],
                                        open=data["Open"], 
                                        high=data["High"],
                                        low=data["Low"], 
                                        close=data["Close"])])
figure.update_layout(title = "Bitcoin Price Analysis", 
                     xaxis_rangeslider_visible=False)
figure.show()

Veri setindeki Kapatma sütunu tahmin etmemiz gereken değerleri içerir. Öyleyse, verilerdeki Kapatma sütunuyla ilgili tüm sütunların korelasyonuna bir bakalım

## Kripto Para Fiyat Tahmin Modeli

Kripto para birimlerinin gelecekteki fiyatlarını tahmin etmek, Zaman serisi analizi sorununa dayanmaktadır. Python'daki AutoTS kütüphanesi, zaman serisi analizi için en iyi kütüphanelerden biridir.Önümüzdeki 30 güne ait Bitcoin fiyatlarını tahmin etmek için AutoTS kütüphanesini kullanacağız.

In [7]:
pip install AutoTS

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 851.0/851.0 kB 4.1 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [8]:
from autots import AutoTS
model = AutoTS(forecast_length=30, frequency='infer', ensemble='simple')
model = model.fit(data, date_col='Date', value_col='Close', id_col=None)
prediction = model.predict()
forecast = prediction.forecast
print(forecast)

Using 4 cpus for n_jobs.
Data frequency is: D, used frequency is: D
Model Number: 1 with model ARIMA in generation 0 of 20
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           17     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.62610D+03    |proj g|=  3.82924D+03

At iterate    1    f=  9.71139D+00    |proj g|=  2.76317D+00


 This problem is unconstrained.



At iterate    2    f=  9.51614D+00    |proj g|=  2.29893D+00

At iterate    3    f=  9.14219D+00    |proj g|=  1.34645D+00

At iterate    4    f=  8.94456D+00    |proj g|=  7.18236D-01

At iterate    5    f=  8.79430D+00    |proj g|=  2.41387D-01

At iterate    6    f=  8.67982D+00    |proj g|=  1.75560D-01

At iterate    7    f=  8.55195D+00    |proj g|=  1.18025D-01

At iterate    8    f=  8.52210D+00    |proj g|=  2.43976D-01

At iterate    9    f=  8.46892D+00    |proj g|=  8.41634D-02

At iterate   10    f=  8.46135D+00    |proj g|=  4.58486D-02

At iterate   11    f=  8.45396D+00    |proj g|=  3.09553D-02

At iterate   12    f=  8.45095D+00    |proj g|=  2.28849D-02

At iterate   13    f=  8.44791D+00    |proj g|=  3.24070D-02

At iterate   14    f=  8.44359D+00    |proj g|=  2.28349D-02

At iterate   15    f=  8.44247D+00    |proj g|=  2.69023D-02

At iterate   16    f=  8.44168D+00    |proj g|=  2.96457D-02

At iterate   17    f=  8.44124D+00    |proj g|=  9.43330D-03

At iter

/opt/anaconda3/lib/python3.11/site-packages/sklearn/svm/_classes.py:31: FutureWarning:

The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.

/opt/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning:

Liblinear failed to converge, increase the number of iterations.

/opt/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html



Model Number: 8 with model DatepartRegression in generation 0 of 20
Epoch 1/50


...

📈 6 - Ensemble with avg smape 4.75: 
Model Number: 7 of 8 with model Ensemble for Validation 3


/opt/anaconda3/lib/python3.11/site-packages/sklearn/neighbors/_classification.py:238: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



7 - Ensemble with avg smape 5.1: 
Model Number: 8 of 8 with model Ensemble for Validation 3


/opt/anaconda3/lib/python3.11/site-packages/sklearn/neighbors/_classification.py:238: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



8 - Ensemble with avg smape 5.3: 
                   Close
2024-10-24  66428.382015
2024-10-25  66410.895409
2024-10-26  66415.854293
2024-10-27  66368.993503
2024-10-28  66576.142430
2024-10-29  66418.534209
2024-10-30  66384.744566
2024-10-31  66350.769212
2024-11-01  67509.323564
2024-11-02  67509.842455
2024-11-03  67458.548481
2024-11-04  67661.271163
2024-11-05  67499.243766
2024-11-06  67461.042148
2024-11-07  67422.662149
2024-11-08  67396.331686
2024-11-09  67392.460980
2024-11-10  67336.785127
2024-11-11  67535.133778
2024-11-12  67368.740326
2024-11-13  67326.180758
2024-11-14  67283.451041
2024-11-15  67252.779223
2024-11-16  67244.575651
2024-11-17  67184.575548
2024-11-18  67378.608693
2024-11-19  67207.908607
2024-11-20  67161.051403
2024-11-21  67114.033175
2024-11-22  67079.082097


# Sonuç:

Alım satım, herhangi bir kripto paranın fiyatında bir değişikliğe neden olur, ancak alım satım eğilimleri birçok faktöre bağlıdır. Kripto para fiyat tahmini için makine öğrenimini kullanmak, yalnızca insanların kripto paralarını alıp satmadan önce gördükleri geçmiş fiyatlar nedeniyle fiyatların değiştiği durumlarda işe yarayabilir.